In [1]:
import pandas as pd
import numpy as np
import time

/home/coea/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
# # data = pd.read_csv("D:/Udler downloads/train.csv")
data = pd.read_csv("../../../datasets/Churn_Modelling.csv")

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [6]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [9]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [11]:
def load_and_standardize_data(path):
    # read in from csv
    df = pd.read_csv(path, sep=',')
    # replace nan with -99
    # df = df.fillna(-99)
    y = df['Exited']
    df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
       'Gender','Exited'],axis=1,inplace=True)
    xtrain,xtest,ytrain,ytest = train_test_split(df,y,stratify = y,test_size = 0.3,random_state = 4)
    # df = df.values.reshape(-1, df.shape[1]).astype('float32')
    # randomly split

    # standardize values
#     scaler = preprocessing.StandardScaler()
    scaler=preprocessing.MinMaxScaler()
    xtrain = scaler.fit_transform(xtrain)
    xtest= scaler.transform(xtest)
    return xtrain,xtest,scaler

In [12]:
from torch.utils.data import Dataset, DataLoader
class DataBuilder(Dataset):
    def __init__(self, path, train=True):
        self.X_train, self.X_test, self.standardizer = load_and_standardize_data(DATA_PATH)
        if train:
            # self.x = torch.from_numpy(self.X_train)
            self.x = torch.FloatTensor(self.X_train)
            self.len=self.x.shape[0]
        else:
            # self.x = torch.from_numpy(self.X_test)
            self.x = torch.FloatTensor(self.X_test)
            self.len=self.x.shape[0]
        del self.X_train
        del self.X_test
    def __getitem__(self,index):
        return self.x[index]
    def __len__(self):
        return self.len

In [13]:
traindata_set=DataBuilder(DATA_PATH, train=True)
testdata_set=DataBuilder(DATA_PATH, train=False)

trainloader=DataLoader(dataset=traindata_set,batch_size=20000)
testloader=DataLoader(dataset=testdata_set,batch_size=20000)

In [14]:
type(trainloader.dataset.x), type(testloader.dataset.x)

(torch.Tensor, torch.Tensor)

In [15]:
trainloader.dataset.x.shape, testloader.dataset.x.shape

(torch.Size([7000, 8]), torch.Size([3000, 8]))

In [16]:
trainloader.dataset.x

tensor([[0.8700, 0.2162, 0.9000,  ..., 1.0000, 1.0000, 0.3430],
        [0.8600, 0.2568, 1.0000,  ..., 1.0000, 0.0000, 0.6317],
        [0.6340, 0.2432, 0.3000,  ..., 1.0000, 1.0000, 0.9344],
        ...,
        [0.3860, 0.2568, 0.3000,  ..., 1.0000, 1.0000, 0.3946],
        [0.4180, 0.3243, 0.7000,  ..., 1.0000, 1.0000, 0.9504],
        [0.7880, 0.3378, 1.0000,  ..., 0.0000, 1.0000, 0.1211]])

In [17]:
testloader.dataset.x

tensor([[0.7860, 0.2973, 0.8000,  ..., 1.0000, 0.0000, 0.4745],
        [0.2760, 0.0541, 0.6000,  ..., 1.0000, 1.0000, 0.3320],
        [0.5320, 0.3378, 0.3000,  ..., 1.0000, 0.0000, 0.0938],
        ...,
        [0.5060, 0.1757, 0.8000,  ..., 1.0000, 1.0000, 0.8498],
        [0.5540, 0.3514, 0.6000,  ..., 0.0000, 0.0000, 0.1765],
        [0.5280, 0.2432, 0.1000,  ..., 1.0000, 0.0000, 0.7308]])

In [18]:
latent_dims = 2
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

## WNN Functions

In [153]:
1/np.sqrt(3)

0.5773502691896258

In [164]:


def ggw(x):
    return torch.sin(3.0 * x)+ torch.sin(0.3 * x)+  torch.sin(0.03 * x)

def morlet_wavelet(x):
#     print(x)
    return (torch.cos(1.75 * x)) * torch.exp(-x**2/2)

def gaussian_wavelet(x,sigma = 1, omega = 0.5 ):
    return torch.exp(-0.5 * (x ** 2) / (sigma** 2)) * torch.cos(omega * x)

def mexhican_hat_wavelet(x, sigma=2, omega=0.577):
    return (torch.pi** (-0.25)*sigma*omega)*(1-x**2)*(torch.exp(-0.5 * (-x**2))) 

def shannon(x):
    return torch.sin(torch.pi *(x-0.5)) - torch.sin(2*(x-0.5) * torch.pi) *(1/torch.pi*(x-0.5))

In [167]:
#shannon(torch.tensor(3.4))

In [22]:
import math
import torch
import torch.nn as nn
import numpy as np

In [23]:
import random

In [24]:
rx=torch.randn(23,50)
weights = torch.Tensor(rx)
print("weights_Encoder",weights)
print("weights_Encoder_shape",weights.shape)

weights_Encoder tensor([[ 0.3042, -0.2151,  1.3061,  ...,  0.6884, -0.7679,  0.7431],
        [ 0.2470, -0.3439,  0.3487,  ...,  1.0744, -1.1642,  0.7968],
        [ 1.4766, -0.4168,  0.6072,  ..., -0.6146,  0.7204,  0.7274],
        ...,
        [-1.0147,  0.2563, -0.0244,  ..., -1.3770,  0.8217,  0.9373],
        [-0.6665, -0.5075,  0.3135,  ...,  0.6813, -0.9960,  0.2647],
        [-1.0920, -0.9895, -1.2209,  ...,  0.5649,  0.1554,  0.3669]])
weights_Encoder_shape torch.Size([23, 50])


In [25]:
weights.t().shape

torch.Size([50, 23])

In [26]:
    x = torch.randn(50)
    torch.Tensor(x).reshape(1,-1)

tensor([[-0.1538, -1.5455, -0.2139,  1.9326,  0.4411,  0.0682,  0.3072, -0.3687,
         -1.0491,  0.0699,  0.5007, -0.4523,  1.0470, -0.9542, -0.5328, -0.4560,
          1.5876,  0.2429,  0.3439, -1.6507, -0.7070, -1.0980,  0.9716, -0.0972,
         -1.5193,  1.3958, -0.8272, -0.1500,  0.3197, -0.3058,  1.8306,  0.4494,
          0.1387,  0.0313,  0.3202, -0.3924, -0.4982, -0.4481,  0.0163, -0.2291,
          1.7649, -1.2005, -0.7354,  0.7836, -0.9654,  0.3931,  0.1707, -1.7513,
         -1.2940, -0.9980]])

In [27]:
x[0]

tensor(-0.1538)

In [28]:
from torch.nn.functional import linear

class WnnLayer(nn.Module):

    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out

        #random_seed = 1
        # torch.manual_seed(random_seed)

        rx=torch.randn(size_in, size_out)
        weights = torch.Tensor(rx)
#         print('weights',weights.shape)

        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
#         self.lr=lr
        # print(self.weights)
        # print((self.weights).shape)
        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)
        drx=torch.randn(self.size_out)
        self.dilation = torch.Tensor(drx)

        self.dilation.requires_grad=True
        self.dilation=nn.Parameter(self.dilation)
        trx=torch.randn(self.size_out)
        self.translation = torch.Tensor(trx)

        self.translation.requires_grad=True
        self.translation=nn.Parameter(self.translation)

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
#         self.lr=lr
        # nn.init.uniform_(self.bias, -bound, bound)  # bias init
        # self.dilation = nn.init.uniform_(self.dilation)
        # self.translation = nn.init.uniform_(self.translation)

    def forward(self, x):
#         print('x',x.shape)
        x1=x.reshape(1,-1)
#         print('x1.shape',x1.shape)
#         print(x1.shape,self.weights.t().shape)
        w_times_x= torch.mm(x, self.weights)
# #         print('w_timesx.shape',w_times_x.shape)

        mean=torch.mean(self.dilation)
        std=torch.std(self.dilation)
        #self.dilation = torch.nn.Parameter(torch.div(torch.sub(self.dilation,mean),std))


        mean=torch.mean(self.translation)
        std=torch.std(self.translation)
        #self.translation = torch.nn.Parameter(torch.div(torch.sub(self.translation,mean),std))

        ###################################################print(self.translation)

        #hid = torch.div(torch.sub(w_times_x , self.dilation_a),self.translation_b)
#         print(w_times_x.shape,self.dilation.shape)

        self.dilation_a=self.dilation.reshape(1,-1)
#         print('dilation.shape',self.dilation.shape)
        self.translation_b=self.translation.reshape(1,-1)
#         print('translation.shape',self.translation.shape)

        hid = torch.div(torch.sub(w_times_x , self.dilation_a),self.translation_b)
#         hid.reshape(-1,1)
#         print('hid',hid.shape)

        hid[hid!= hid]=0.0
#         print(hid.shape)
        return hid
        # return torch.add(w_times_x, self.bias)  # w times x + b

In [29]:
from torch.nn.functional import linear

class WnnLayer1(nn.Module):

    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        random_seed = 1
        #torch.manual_seed(random_seed)
        rx=torch.randn(size_out, size_in)
        weights = torch.Tensor(rx)
       # print("weights_Decoder",weights)
#         print("weights_Decoder_shape",weights.shape)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
        # print(self.weights)
        # print((self.weights).shape)
        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)
        drx=torch.randn(self.size_out)
        self.dilation = torch.Tensor(drx)
        self.dilation.requires_grad=True
        self.dilation=nn.Parameter(self.dilation)
        trx=torch.randn(self.size_out)
        self.translation = torch.Tensor(trx)
        self.translation.requires_grad=True
        self.translation=nn.Parameter(self.translation)
#         self.lr=lr

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)


    def forward(self, x):
#         print('x.shape',x.shape)

        x1=x.reshape(1,-1)
#         print('x1 and weights shapes: ',x1.shape,self.weights.t().shape)
        w_times_x= torch.mm(x, self.weights.t())
#         print('w_times_x.shape',w_times_x.shape)

#         print(f'w_times_x and dilation shapes: ',w_times_x.shape,self.dilation.shape)
        mean=torch.mean(self.dilation)
        std=torch.std(self.dilation)
        #self.dilation = torch.nn.Parameter(torch.div(torch.sub(self.dilation,mean),std))


        mean=torch.mean(self.translation)
        std=torch.std(self.translation)
        #self.translation = torch.nn.Parameter(torch.div(torch.sub(self.translation,mean),std))


        self.dilation_a=self.dilation.reshape(1,-1)
#         print('dilation.shape',self.dilation.shape)

        self.translation_b=self.translation.reshape(1,-1)
#         print('translation.shape',self.translation.shape)


        hid = torch.div(torch.sub(w_times_x , self.dilation_a),self.translation_b)
#         print('hid.shape',hid.shape)

#         print(f'hidden values shapes: ',hid.shape)
        return hid
        # return torch.add(w_times_x, self.bias)  # w times x + b

## Encoder

In [527]:
## GGW

class Encoder2(nn.Module):
    def __init__(self,input_no,latent_dims,fn,H2 = 10,H3=15):
        super(Encoder2,self).__init__()
        self.linear1 = WnnLayer(input_no,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        self.linear3 = nn.Linear(H3,latent_dims)
        self.binorm3= nn.BatchNorm1d(num_features = latent_dims)
        self.linear4= nn.Linear(H3 , latent_dims)
        self.binorm4= nn.BatchNorm1d(num_features = latent_dims)
        self.relu =nn.ReLU()
        self.fn=fn
        #act=['morlet','ggw','mexi','shan','gaus']
    def forward(self, x):
        if self.fn=='ggw':
            lin1 = ggw(self.binorm1(self.linear1(x)))
            lin2 = ggw(self.binorm2(self.linear2(lin1)))
            Mu = ggw(self.binorm3(self.linear3(lin2)))
            sigma = ggw(self.binorm4(self.linear4(lin2)))
        if self.fn=='morlet':
            lin1 = morlet_wavelet(self.binorm1(self.linear1(x)))
            lin2 = morlet_wavelet(self.binorm2(self.linear2(lin1)))
            Mu = morlet_wavelet(self.binorm3(self.linear3(lin2)))
            sigma = morlet_wavelet(self.binorm4(self.linear4(lin2)))
        if self.fn=='mexi':
            lin1 = mexhican_hat_wavelet(self.binorm1(self.linear1(x)))
            lin2 = mexhican_hat_wavelet(self.binorm2(self.linear2(lin1)))
            Mu = mexhican_hat_wavelet(self.binorm3(self.linear3(lin2)))
            sigma = mexhican_hat_wavelet(self.binorm4(self.linear4(lin2)))
        if self.fn=='shan':
            lin1 = shannon(self.binorm1(self.linear1(x)))
            lin2 = shannon(self.binorm2(self.linear2(lin1)))
            Mu = shannon(self.binorm3(self.linear3(lin2)))
            sigma = shannon(self.binorm4(self.linear4(lin2)))
        if self.fn=='gaus':
            lin1 = gaussian_wavelet(self.binorm1(self.linear1(x)))
            lin2 = gaussian_wavelet(self.binorm2(self.linear2(lin1)))
            Mu = gaussian_wavelet(self.binorm3(self.linear3(lin2)))
            sigma = gaussian_wavelet(self.binorm4(self.linear4(lin2)))

        return Mu,sigma

class Encoder3(nn.Module):
    def __init__(self,input_no,latent_dims,fn,H2 = 10,H3=15,H4=20):
        super(Encoder3,self).__init__()
        self.linear1 = WnnLayer(input_no,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        ##
        self.linear3 = WnnLayer(H3,H4)
        self.binorm3 = nn.BatchNorm1d(num_features = H4)
        self.linear4 = nn.Linear(H4,latent_dims)
        ##
        self.binorm4= nn.BatchNorm1d(num_features = latent_dims)
        self.linear5= nn.Linear(H4 , latent_dims)
        self.binorm5= nn.BatchNorm1d(num_features = latent_dims)
        self.relu =nn.ReLU()
        self.fn=fn
        
    def forward(self, x):
        if self.fn=='ggw':
            lin1 = ggw(self.binorm1(self.linear1(x)))
            lin2 = ggw(self.binorm2(self.linear2(lin1)))
            lin3 = ggw(self.binorm3(self.linear3(lin2)))
            Mu = ggw(self.binorm4(self.linear4(lin3)))
            sigma = ggw(self.binorm5(self.linear5(lin3)))
        if self.fn=='morlet':
            lin1 = morlet_wavelet(self.binorm1(self.linear1(x)))
            lin2 = morlet_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = morlet_wavelet(self.binorm3(self.linear3(lin2)))
            Mu = morlet_wavelet(self.binorm4(self.linear4(lin3)))
            sigma = morlet_wavelet(self.binorm5(self.linear5(lin3)))
        if self.fn=='mexi':
            lin1 = mexhican_hat_wavelet(self.binorm1(self.linear1(x)))
            lin2 = mexhican_hat_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = mexhican_hat_wavelet(self.binorm3(self.linear3(lin2)))
            Mu = mexhican_hat_wavelet(self.binorm4(self.linear4(lin3)))
            sigma = mexhican_hat_wavelet(self.binorm5(self.linear5(lin3)))
        if self.fn=='shan':
            lin1 = shannon(self.binorm1(self.linear1(x)))
            lin2 = shannon(self.binorm2(self.linear2(lin1)))
            lin3 = shannon(self.binorm3(self.linear3(lin2)))
            Mu = shannon(self.binorm4(self.linear4(lin3)))
            sigma = shannon(self.binorm5(self.linear5(lin3)))
        if self.fn=='gaus':
            lin1 = gaussian_wavelet(self.binorm1(self.linear1(x)))
            lin2 = gaussian_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = gaussian_wavelet(self.binorm3(self.linear3(lin2)))
            Mu = gaussian_wavelet(self.binorm4(self.linear4(lin3)))
            sigma = gaussian_wavelet(self.binorm5(self.linear5(lin3)))

        return Mu,sigma
    
class Encoder4(nn.Module):
    def __init__(self,input_no,latent_dims,fn,H2 = 10,H3=15,H4=20,H5=22):
        super(Encoder4,self).__init__()
        self.linear1 = WnnLayer(input_no,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        ##
        self.linear3 = WnnLayer(H3,H4)
        self.binorm3 = nn.BatchNorm1d(num_features = H4)
        self.linear4 = nn.Linear(H4,latent_dims)
        ##
        ##
        self.linear4 = WnnLayer(H4,H5)
        self.binorm4 = nn.BatchNorm1d(num_features = H5)
        self.linear5 = nn.Linear(H5,latent_dims)
        ##
        self.binorm5= nn.BatchNorm1d(num_features = latent_dims)
        self.linear6= nn.Linear(H5 , latent_dims)
        self.binorm6= nn.BatchNorm1d(num_features = latent_dims)
        self.relu =nn.ReLU()
        self.fn=fn
        
    def forward(self, x):
        if self.fn=='ggw':
            lin1 = ggw(self.binorm1(self.linear1(x)))
            lin2 = ggw(self.binorm2(self.linear2(lin1)))
            lin3 = ggw(self.binorm3(self.linear3(lin2)))
            lin4 = ggw(self.binorm4(self.linear4(lin3)))
            Mu = ggw(self.binorm5(self.linear5(lin4)))
            sigma = ggw(self.binorm6(self.linear6(lin4)))
        if self.fn=='morlet':
            lin1 = morlet_wavelet(self.binorm1(self.linear1(x)))
            lin2 = morlet_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = morlet_wavelet(self.binorm3(self.linear3(lin2)))
            lin4 = morlet_wavelet(self.binorm4(self.linear4(lin3)))
            Mu = morlet_wavelet(self.binorm5(self.linear5(lin4)))
            sigma = morlet_wavelet(self.binorm6(self.linear6(lin4)))
        if self.fn=='mexi':
            lin1 = mexhican_hat_wavelet(self.binorm1(self.linear1(x)))
            lin2 = mexhican_hat_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = mexhican_hat_wavelet(self.binorm3(self.linear3(lin2)))
            lin4 = mexhican_hat_wavelet(self.binorm4(self.linear4(lin3)))
            Mu = mexhican_hat_wavelet(self.binorm5(self.linear5(lin4)))
            sigma = mexhican_hat_wavelet(self.binorm6(self.linear6(lin4)))
        if self.fn=='shan':
            lin1 = shannon(self.binorm1(self.linear1(x)))
            lin2 = shannon(self.binorm2(self.linear2(lin1)))
            lin3 = shannon(self.binorm3(self.linear3(lin2)))
            lin4 = shannon(self.binorm4(self.linear4(lin3)))
            Mu = shannon(self.binorm5(self.linear5(lin4)))
            sigma = shannon(self.binorm6(self.linear6(lin4)))
        if self.fn=='gaus':
            lin1 = gaussian_wavelet(self.binorm1(self.linear1(x)))
            lin2 = gaussian_wavelet(self.binorm2(self.linear2(lin1)))
            lin3 = gaussian_wavelet(self.binorm3(self.linear3(lin2)))
            lin4 = gaussian_wavelet(self.binorm4(self.linear4(lin3)))
            Mu = gaussian_wavelet(self.binorm5(self.linear5(lin4)))
            sigma = gaussian_wavelet(self.binorm6(self.linear6(lin4)))

        return Mu,sigma

In [528]:
class Decoder2(nn.Module):
    def __init__(self,latent_dims,fn,H2=15,H3=10):
        super(Decoder2, self).__init__()
#         self.linear1 = nn.Linear(latent_dims, 3400)
        self.linear1 = WnnLayer1(latent_dims,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer1(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        self.linear3 = WnnLayer1(H3,8)
        self.binorm3 = nn.BatchNorm1d(num_features = 8)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fn=fn

    def forward(self, z):
        #print(z)
        # z = meyer_new(self.linear1(z))
        if self.fn=='ggw':
            z1 = ggw(self.binorm1(self.linear1(z)))
            linbn2 = ggw(self.binorm2(self.linear2(z1)))
            linbn3 = ggw(self.binorm3(self.linear3(linbn2)))
        if self.fn=='morlet':
            z1 = morlet_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = morlet_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = morlet_wavelet(self.binorm3(self.linear3(linbn2)))
        if self.fn=='mexi':
            z1 = mexhican_hat_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = mexhican_hat_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = mexhican_hat_wavelet(self.binorm3(self.linear3(linbn2)))
        if self.fn=='shan':
            z1 = shannon(self.binorm1(self.linear1(z)))
            linbn2 = shannon(self.binorm2(self.linear2(z1)))
            linbn3 = shannon(self.binorm3(self.linear3(linbn2)))
        if self.fn=='gaus':
            z1 = gaussian_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = gaussian_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = gaussian_wavelet(self.binorm3(self.linear3(linbn2)))

        return linbn3


class Decoder3(nn.Module):
    def __init__(self,latent_dims,fn,H2=20,H3=15,H4=10):
        super(Decoder3, self).__init__()
#         self.linear1 = nn.Linear(latent_dims, 3400)
        self.linear1 = WnnLayer1(latent_dims,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer1(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        ##
        self.linear3 = WnnLayer1(H3,H4)
        self.binorm3 = nn.BatchNorm1d(num_features = H4)
        ##
        self.linear4 = WnnLayer1(H4,8)
        self.binorm4 = nn.BatchNorm1d(num_features = 8)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fn=fn

    def forward(self, z):
        #print(z)
        # z = meyer_new(self.linear1(z))
        if self.fn=='ggw':
            z1 = ggw(self.binorm1(self.linear1(z)))
            linbn2 = ggw(self.binorm2(self.linear2(z1)))
            linbn3 = ggw(self.binorm3(self.linear3(linbn3)))
            linbn4 = ggw(self.binorm4(self.linear4(linbn3)))
        
        if self.fn=='morlet':
            z1 = morlet_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = morlet_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = morlet_wavelet(self.binorm3(self.linear3(linbn2)))
            linbn4 = morlet_wavelet(self.binorm4(self.linear4(linbn3)))
        
        if self.fn=='mexi':
            z1 = mexhican_hat_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = mexhican_hat_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = mexhican_hat_wavelet(self.binorm3(self.linear3(linbn3)))
            linbn4 = mexhican_hat_wavelet(self.binorm4(self.linear4(linbn3)))
        
        if self.fn=='shan':
            z1 = shannon(self.binorm1(self.linear1(z)))
            linbn2 = shannon(self.binorm2(self.linear2(z1)))
            linbn3 = shannon(self.binorm3(self.linear3(linbn2)))
            linbn4 = shannon(self.binorm4(self.linear4(linbn3)))
        
        if self.fn=='gaus':
            z1 = gaussian_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = gaussian_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = gaussian_wavelet(self.binorm3(self.linear3(linbn3)))
            linbn4 = gaussian_wavelet(self.binorm4(self.linear4(linbn3)))

        return linbn4

class Decoder4(nn.Module):
    def __init__(self,latent_dims,fn,H2=22,H3=20,H4=5,H5=10):
        super(Decoder4, self).__init__()
#         self.linear1 = nn.Linear(latent_dims, 3400)
        self.linear1 = WnnLayer1(latent_dims,H2)
        self.binorm1 = nn.BatchNorm1d(num_features = H2)
        self.linear2 = WnnLayer1(H2,H3)
        self.binorm2 = nn.BatchNorm1d(num_features = H3)
        ##
        self.linear3 = WnnLayer1(H3,H4)
        self.binorm3 = nn.BatchNorm1d(num_features = H4)
        ##
        ##
        self.linear4 = WnnLayer1(H4,H5)
        self.binorm4 = nn.BatchNorm1d(num_features = H5)
        ##
        self.linear5 = WnnLayer1(H5,8)
        self.binorm5 = nn.BatchNorm1d(num_features = 8)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fn=fn

    def forward(self, z):
        #print(z)
        # z = meyer_new(self.linear1(z))
        if self.fn=='ggw':
            z1 = ggw(self.binorm1(self.linear1(z)))
            linbn2 = ggw(self.binorm2(self.linear2(z1)))
            linbn3 = ggw(self.binorm3(self.linear3(linbn3)))
            linbn4 = ggw(self.binorm4(self.linear4(linbn3)))
            linbn5 = ggw(self.binorm5(self.linear5(linbn4)))
        
        if self.fn=='morlet':
            z1 = morlet_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = morlet_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = morlet_wavelet(self.binorm3(self.linear3(linbn2)))
            linbn4 = morlet_wavelet(self.binorm4(self.linear4(linbn3)))
            linbn5 = morlet_wavelet(self.binorm5(self.linear5(linbn4)))
        
        if self.fn=='mexi':
            z1 = mexhican_hat_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = mexhican_hat_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = mexhican_hat_wavelet(self.binorm3(self.linear3(linbn3)))
            linbn4 = mexhican_hat_wavelet(self.binorm4(self.linear4(linbn3)))
            linbn5 = mexhican_hat_wavelet(self.binorm5(self.linear5(linbn4)))
        
        if self.fn=='shan':
            z1 = shannon(self.binorm1(self.linear1(z)))
            linbn2 = shannon(self.binorm2(self.linear2(z1)))
            linbn3 = shannon(self.binorm3(self.linear3(linbn2)))
            linbn4 = shannon(self.binorm4(self.linear4(linbn3)))
            linbn5 = shannon(self.binorm5(self.linear5(linbn4)))
        
        if self.fn=='gaus':
            z1 = gaussian_wavelet(self.binorm1(self.linear1(z)))
            linbn2 = gaussian_wavelet(self.binorm2(self.linear2(z1)))
            linbn3 = gaussian_wavelet(self.binorm3(self.linear3(linbn3)))
            linbn4 = gaussian_wavelet(self.binorm4(self.linear4(linbn3)))
            linbn5 = gaussian_wavelet(self.binorm5(self.linear5(linbn4)))
        

        return linbn5

In [529]:
class sampling_vector(nn.Module):
    def __init__(self, z,H2):
        super().__init__()
        self.z = z
        self.fc3 = nn.Linear(self.z,H2)
        self.fc_bn3 = nn.BatchNorm1d(H2)
#         self.fc4 = nn.Linear(latent_dims, H2)
#         self.fc_bn4 = nn.BatchNorm1d(H2)
        self.relu = nn.ReLU()
    def forward(self,z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        return fc3

In [530]:
def reparameterize(mu,sigma):

        std = sigma.mul(0.5).exp_()
        eps = Variable(std.data.new(std.size())).normal_()
#         print("reparameter",eps.mul(std).add_(mu))
        return eps.mul(std).add_(mu)


In [531]:
class VariationalAutoencoder2(nn.Module):
    def __init__(self,D_in,latent_dims,fn,H2 = 23):
        super(VariationalAutoencoder2, self).__init__()
        self.encoder = Encoder2(D_in,latent_dims,fn)
        self.decoder = Decoder2(latent_dims,fn)

    def forward(self, x):
        mu,sigma = self.encoder(x)
        z = reparameterize(mu,sigma)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decoder(reparameterize(mu,sigma)),mu,sigma

class VariationalAutoencoder3(nn.Module):
    def __init__(self,D_in,latent_dims,fn,H2 = 23):
        super(VariationalAutoencoder3, self).__init__()
        self.encoder = Encoder3(D_in,latent_dims,fn)
        self.decoder = Decoder3(latent_dims,fn)

    def forward(self, x):
        mu,sigma = self.encoder(x)
        z = reparameterize(mu,sigma)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decoder(reparameterize(mu,sigma)),mu,sigma

class VariationalAutoencoder4(nn.Module):
    def __init__(self,D_in,latent_dims,fn,H2 = 23):
        super(VariationalAutoencoder4, self).__init__()
        self.encoder = Encoder4(D_in,latent_dims,fn)
        self.decoder = Decoder4(latent_dims,fn)

    def forward(self, x):
        mu,sigma = self.encoder(x)
        z = reparameterize(mu,sigma)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decoder(reparameterize(mu,sigma)),mu,sigma

In [532]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")

    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [533]:
epochs = 1500
log_interval = 50
val_losses = []
train_losses = []
test_losses = []


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if epoch % 100 == 0:
            print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))
        
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % 100 == 0:
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))

In [534]:
loss_mse = customLoss()

In [535]:
D_in = trainloader.dataset.x.shape[1]
model = VariationalAutoencoder4(D_in,2,'shan').to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# optimizer = optim.SGD(model.parameters(), lr=0.09,momentum=0.01)
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

In [537]:
start = time.time()
epochs=300
for epoch in range(1, epochs + 1):
    train(epoch)
#     test(epoch)
elapsed_time = time.time()-start
print("total_time taken is :",elapsed_time)
#     test(epoch)

====> Epoch: 100 Average training loss: 6.7088
====> Epoch: 200 Average training loss: 6.5969
====> Epoch: 300 Average training loss: 6.3176
total_time taken is : 13.795526266098022


In [538]:
print(train_losses[len(train_losses)-1])

6.317590401785714


## HYPER-PARAMETER TUNING

In [578]:
global model
global optimizer

model = VariationalAutoencoder4(D_in,2,'shan').to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

In [579]:
epochs=[1800]
hl=[2,3,4]
lr=[0.0001,0.01,0.001]
mm=[0.001,0.01]
act=['morlet','ggw','mexi','shan','gaus']
opt=['adam','adagrad','sgd']
ldm=[2,3,4]

min_loss=10000
MODEL=None
BESTCOMB=None

for e in epochs:
    for h in hl:
        for l in lr:
            for m in mm:
                for a in act:
                    for o in opt:
                        for ld in ldm:
                            
                            D_in = trainloader.dataset.x.shape[1]
                            
                            if hl==2:
                                model = VariationalAutoencoder2(D_in,ld,a).to(device)
                            if hl==3:
                                model = VariationalAutoencoder3(D_in,ld,a).to(device)
                            if hl==4:
                                model = VariationalAutoencoder4(D_in,ld,a).to(device)
                            
                            if opt=='adam':
                                optimizer = optim.Adam(model.parameters(), lr=l)
                            if opt=='sgd':
                                optimizer = optim.SGD(model.parameters(), lr=l,momentum=m)
                            if opt=='adagrad':
                                optimizer = torch.optim.Adagrad(model.parameters(), lr = l)
                            
                            start = time.time()
                            for epoch in range(1, e+1):
                                train(epoch)
                            #     test(epoch)
                            elapsed_time = time.time()-start
                            #print("total_time taken is :",elapsed_time)

                            cur_loss=train_losses[len(train_losses)-1]
                            if cur_loss < min_loss:
                                min_loss=cur_loss
                                MODEL=model
                                BESTCOMB=[e,h,l,m,a,o,ld]
                            
                            
                            ## junk --ignore
                            model = VariationalAutoencoder4(D_in,2,'shan').to(device)
                            optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

====> Epoch: 100 Average training loss: 6.7463
====> Epoch: 200 Average training loss: 6.4541
====> Epoch: 300 Average training loss: 5.9567
====> Epoch: 400 Average training loss: 5.7686
====> Epoch: 500 Average training loss: 5.5655
====> Epoch: 600 Average training loss: 5.4035
====> Epoch: 700 Average training loss: 5.3328
====> Epoch: 800 Average training loss: 5.2518
====> Epoch: 900 Average training loss: 5.0926
====> Epoch: 1000 Average training loss: 4.9953
====> Epoch: 1100 Average training loss: 4.8845
====> Epoch: 1200 Average training loss: 4.7315
====> Epoch: 1300 Average training loss: 4.6060
====> Epoch: 1400 Average training loss: 4.5275
====> Epoch: 1500 Average training loss: 4.4714
====> Epoch: 1600 Average training loss: 4.4449
====> Epoch: 1700 Average training loss: 4.3784
====> Epoch: 1800 Average training loss: 4.3850
====> Epoch: 100 Average training loss: 6.8475
====> Epoch: 200 Average training loss: 6.3200
====> Epoch: 300 Average training loss: 6.0446
====

====> Epoch: 1200 Average training loss: 4.9904
====> Epoch: 1300 Average training loss: 4.9496
====> Epoch: 1400 Average training loss: 4.9657
====> Epoch: 1500 Average training loss: 4.9292
====> Epoch: 1600 Average training loss: 4.8852
====> Epoch: 1700 Average training loss: 4.8799
====> Epoch: 1800 Average training loss: 4.8508
====> Epoch: 100 Average training loss: 7.2274
====> Epoch: 200 Average training loss: 6.8593
====> Epoch: 300 Average training loss: 6.6205
====> Epoch: 400 Average training loss: 6.3808
====> Epoch: 500 Average training loss: 6.1148
====> Epoch: 600 Average training loss: 5.8630
====> Epoch: 700 Average training loss: 5.7759
====> Epoch: 800 Average training loss: 5.5530
====> Epoch: 900 Average training loss: 5.5485
====> Epoch: 1000 Average training loss: 5.4720
====> Epoch: 1100 Average training loss: 5.1832
====> Epoch: 1200 Average training loss: 4.9248
====> Epoch: 1300 Average training loss: 4.9111
====> Epoch: 1400 Average training loss: 4.7337
=

====> Epoch: 500 Average training loss: 5.7452
====> Epoch: 600 Average training loss: 5.6222
====> Epoch: 700 Average training loss: 5.5822
====> Epoch: 800 Average training loss: 5.4717
====> Epoch: 900 Average training loss: 5.4283
====> Epoch: 1000 Average training loss: 5.3512
====> Epoch: 1100 Average training loss: 5.3711
====> Epoch: 1200 Average training loss: 5.3432
====> Epoch: 1300 Average training loss: 5.3545
====> Epoch: 1400 Average training loss: 5.2525
====> Epoch: 1500 Average training loss: 5.3039
====> Epoch: 1600 Average training loss: 5.2231
====> Epoch: 1700 Average training loss: 5.2349
====> Epoch: 1800 Average training loss: 5.1984
====> Epoch: 100 Average training loss: 7.0531
====> Epoch: 200 Average training loss: 6.4337
====> Epoch: 300 Average training loss: 6.0411
====> Epoch: 400 Average training loss: 5.8945
====> Epoch: 500 Average training loss: 5.7218
====> Epoch: 600 Average training loss: 5.6314
====> Epoch: 700 Average training loss: 5.5428
====

====> Epoch: 1600 Average training loss: 5.1146
====> Epoch: 1700 Average training loss: 5.1104
====> Epoch: 1800 Average training loss: 5.1538
====> Epoch: 100 Average training loss: 7.2971
====> Epoch: 200 Average training loss: 6.8247
====> Epoch: 300 Average training loss: 6.6788
====> Epoch: 400 Average training loss: 6.7054
====> Epoch: 500 Average training loss: 6.4721
====> Epoch: 600 Average training loss: 6.2774
====> Epoch: 700 Average training loss: 6.1240
====> Epoch: 800 Average training loss: 5.9117
====> Epoch: 900 Average training loss: 5.8557
====> Epoch: 1000 Average training loss: 5.7634
====> Epoch: 1100 Average training loss: 5.7192
====> Epoch: 1200 Average training loss: 5.6845
====> Epoch: 1300 Average training loss: 5.5741
====> Epoch: 1400 Average training loss: 5.5961
====> Epoch: 1500 Average training loss: 5.5046
====> Epoch: 1600 Average training loss: 5.5073
====> Epoch: 1700 Average training loss: 5.4723
====> Epoch: 1800 Average training loss: 5.3236
=

====> Epoch: 900 Average training loss: 5.7680
====> Epoch: 1000 Average training loss: 5.7015
====> Epoch: 1100 Average training loss: 5.4270
====> Epoch: 1200 Average training loss: 5.4929
====> Epoch: 1300 Average training loss: 5.3768
====> Epoch: 1400 Average training loss: 5.2901
====> Epoch: 1500 Average training loss: 5.1962
====> Epoch: 1600 Average training loss: 5.1673
====> Epoch: 1700 Average training loss: 5.1366
====> Epoch: 1800 Average training loss: 5.0767
====> Epoch: 100 Average training loss: 6.6394
====> Epoch: 200 Average training loss: 6.1500
====> Epoch: 300 Average training loss: 5.8612
====> Epoch: 400 Average training loss: 5.7675
====> Epoch: 500 Average training loss: 5.5256
====> Epoch: 600 Average training loss: 5.5520
====> Epoch: 700 Average training loss: 5.4972
====> Epoch: 800 Average training loss: 5.5083
====> Epoch: 900 Average training loss: 5.3644
====> Epoch: 1000 Average training loss: 5.3500
====> Epoch: 1100 Average training loss: 5.3732
==

...

====> Epoch: 1700 Average training loss: 5.3129
====> Epoch: 1800 Average training loss: 5.3113
====> Epoch: 100 Average training loss: 6.9534
====> Epoch: 200 Average training loss: 6.5796
====> Epoch: 300 Average training loss: 6.3350
====> Epoch: 400 Average training loss: 6.1842
====> Epoch: 500 Average training loss: 5.9687
====> Epoch: 600 Average training loss: 5.8579
====> Epoch: 700 Average training loss: 5.6955
====> Epoch: 800 Average training loss: 5.4918
====> Epoch: 900 Average training loss: 5.4553
====> Epoch: 1000 Average training loss: 5.3183
====> Epoch: 1100 Average training loss: 5.3026
====> Epoch: 1200 Average training loss: 5.3120
====> Epoch: 1300 Average training loss: 5.2113
====> Epoch: 1400 Average training loss: 5.1771
====> Epoch: 1500 Average training loss: 5.1880
====> Epoch: 1600 Average training loss: 5.0829
====> Epoch: 1700 Average training loss: 5.1117
====> Epoch: 1800 Average training loss: 5.0653
====> Epoch: 100 Average training loss: 7.1114
==

====> Epoch: 1000 Average training loss: 5.3916
====> Epoch: 1100 Average training loss: 5.3757
====> Epoch: 1200 Average training loss: 5.2955
====> Epoch: 1300 Average training loss: 5.3473
====> Epoch: 1400 Average training loss: 5.2414
====> Epoch: 1500 Average training loss: 5.2282
====> Epoch: 1600 Average training loss: 5.1996
====> Epoch: 1700 Average training loss: 5.2202
====> Epoch: 1800 Average training loss: 5.1414
====> Epoch: 100 Average training loss: 7.1970
====> Epoch: 200 Average training loss: 6.8024
====> Epoch: 300 Average training loss: 6.5446
====> Epoch: 400 Average training loss: 6.4342
====> Epoch: 500 Average training loss: 6.3347
====> Epoch: 600 Average training loss: 6.2012
====> Epoch: 700 Average training loss: 6.1417
====> Epoch: 800 Average training loss: 6.0677
====> Epoch: 900 Average training loss: 5.9973
====> Epoch: 1000 Average training loss: 5.8584
====> Epoch: 1100 Average training loss: 5.6998
====> Epoch: 1200 Average training loss: 5.6033
=

====> Epoch: 300 Average training loss: 5.9384
====> Epoch: 400 Average training loss: 5.8657
====> Epoch: 500 Average training loss: 5.7418
====> Epoch: 600 Average training loss: 5.5986
====> Epoch: 700 Average training loss: 5.6342
====> Epoch: 800 Average training loss: 5.5012
====> Epoch: 900 Average training loss: 5.4600
====> Epoch: 1000 Average training loss: 5.3790
====> Epoch: 1100 Average training loss: 5.3690
====> Epoch: 1200 Average training loss: 5.3640
====> Epoch: 1300 Average training loss: 5.3002
====> Epoch: 1400 Average training loss: 5.3187
====> Epoch: 1500 Average training loss: 5.2842
====> Epoch: 1600 Average training loss: 5.2233
====> Epoch: 1700 Average training loss: 5.2699
====> Epoch: 1800 Average training loss: 5.2093
====> Epoch: 100 Average training loss: 7.3328
====> Epoch: 200 Average training loss: 7.0580
====> Epoch: 300 Average training loss: 6.7813
====> Epoch: 400 Average training loss: 6.4345
====> Epoch: 500 Average training loss: 6.3902
====

====> Epoch: 1400 Average training loss: 5.3484
====> Epoch: 1500 Average training loss: 5.3207
====> Epoch: 1600 Average training loss: 5.2733
====> Epoch: 1700 Average training loss: 5.2322
====> Epoch: 1800 Average training loss: 5.0885
====> Epoch: 100 Average training loss: 6.7023
====> Epoch: 200 Average training loss: 6.1837
====> Epoch: 300 Average training loss: 5.9013
====> Epoch: 400 Average training loss: 5.7092
====> Epoch: 500 Average training loss: 5.9154
====> Epoch: 600 Average training loss: 5.7247
====> Epoch: 700 Average training loss: 5.6341
====> Epoch: 800 Average training loss: 5.7612
====> Epoch: 900 Average training loss: 5.6965
====> Epoch: 1000 Average training loss: 5.6821
====> Epoch: 1100 Average training loss: 5.6426
====> Epoch: 1200 Average training loss: 5.5726
====> Epoch: 1300 Average training loss: 5.5940
====> Epoch: 1400 Average training loss: 5.5309
====> Epoch: 1500 Average training loss: 5.5229
====> Epoch: 1600 Average training loss: 5.5084
=

====> Epoch: 700 Average training loss: 5.5677
====> Epoch: 800 Average training loss: 5.4580
====> Epoch: 900 Average training loss: 5.4002
====> Epoch: 1000 Average training loss: 5.3395
====> Epoch: 1100 Average training loss: 5.3256
====> Epoch: 1200 Average training loss: 5.3178
====> Epoch: 1300 Average training loss: 5.2528
====> Epoch: 1400 Average training loss: 5.1813
====> Epoch: 1500 Average training loss: 5.1558
====> Epoch: 1600 Average training loss: 5.1154
====> Epoch: 1700 Average training loss: 5.1361
====> Epoch: 1800 Average training loss: 5.0052
====> Epoch: 100 Average training loss: 6.7345
====> Epoch: 200 Average training loss: 6.2417
====> Epoch: 300 Average training loss: 5.8535
====> Epoch: 400 Average training loss: 5.6297
====> Epoch: 500 Average training loss: 5.5864
====> Epoch: 600 Average training loss: 5.4427
====> Epoch: 700 Average training loss: 5.3087
====> Epoch: 800 Average training loss: 5.2082
====> Epoch: 900 Average training loss: 5.2124
====

In [580]:
BESTCOMB

[1800, 2, 0.001, 0.01, 'mexi', 'adagrad', 2]

In [581]:
MODEL

VariationalAutoencoder4(
  (encoder): Encoder4(
    (linear1): WnnLayer()
    (binorm1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear2): WnnLayer()
    (binorm2): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear3): WnnLayer()
    (binorm3): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear4): WnnLayer()
    (binorm4): BatchNorm1d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear5): Linear(in_features=22, out_features=2, bias=True)
    (binorm5): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear6): Linear(in_features=22, out_features=2, bias=True)
    (binorm6): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (decoder): Decoder4(
    (linear1): WnnLayer1()
    (binorm1): BatchNorm1d(22, eps=1e-05, momentum=0.1, affine=True, track_ru

In [582]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)


In [583]:
x_hat

[tensor([[ 0.7513,  0.2139,  0.7437,  ..., -0.1287,  1.0631, -0.4856],
         [-0.9997, -0.8674, -0.4242,  ...,  0.6514,  0.8381,  0.3852],
         [ 0.9458,  0.0812,  0.4965,  ...,  0.2896,  0.8461,  0.2475],
         ...,
         [ 0.9883,  0.3036,  0.4848,  ...,  0.8122,  0.1094,  0.6189],
         [ 0.7050,  0.1454,  0.7695,  ..., -0.0174,  1.1916, -0.4583],
         [ 1.0268,  0.4991,  0.6377,  ...,  1.0392,  0.0391,  0.7952]])]

In [584]:
d1 = pd.DataFrame(np.array(x_hat[0]))

In [585]:
d1

,0,1,2,3,4,5,6,7
0,0.751310,0.213934,0.743711,-0.573453,-0.888222,-0.128680,1.063090,-0.485571
1,-0.999668,-0.867419,-0.424182,0.474562,0.339953,0.651449,0.838123,0.385234
2,0.945847,0.081211,0.496455,0.986120,-0.070787,0.289577,0.846139,0.247496
3,1.063080,0.249732,0.543527,-0.541635,-0.898730,-0.877791,0.554200,-0.433697
4,0.124483,0.051932,0.755893,0.440277,-0.564288,-0.346430,1.231976,-0.186933
...,...,...,...,...,...,...,...,...
6995,1.016273,0.115567,-0.436659,0.467418,0.331404,0.451333,0.232579,0.520665
6996,1.020711,0.417914,0.172702,-0.972851,-0.273678,0.645902,-0.180840,0.414534
6997,0.988311,0.303569,0.484784,0.238360,0.231577,0.812222,0.109364,0.618921
6998,0.704977,0.145449,0.769528,-0.664398,-0.845569,-0.017420,1.191621,-0.458265


In [586]:
# d2

In [587]:
org =pd.DataFrame(np.array(trainloader.dataset.x))

In [588]:
# pd.DataFrame(np.array(testloader.dataset.x)).mean()


In [589]:
org.std()

0    0.192962
1    0.141110
2    0.287587
3    0.248447
4    0.192987
5    0.456147
6    0.499946
7    0.286641
dtype: float32

In [590]:
syn_test = pd.concat([d1],ignore_index = True)

In [591]:
syn_test.shape

(7000, 8)

In [592]:
syn_test.std()

0    0.478168
1    0.354657
2    0.454520
3    0.517975
4    0.408136
5    0.567058
6    0.496998
7    0.511387
dtype: float32

In [593]:
# d1.to_csv('vae-wnn-yes.csv')

## Evasion Attack

In [598]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(testloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_test = pd.concat([d1],ignore_index = True)

In [599]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, stratify=y, random_state=4)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train,Y_train)

preds = lr.predict(eva_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(X_train,Y_train)

preds =logi.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
parameters = {'criterion' : ["gini", "entropy"],
              'n_estimators': [25,50,100,150,175,200]}
best_model=None
best_score=0.0

for ne in parameters['n_estimators']:
    gb = LGBMClassifier(n_estimators=ne)
    gb.fit(X_train,Y_train)
    y_predg=gb.predict(X_test)

    accuracy=accuracy_score(Y_test,y_predg)
    auc=roc_auc_score(Y_test,y_predg)
    # print('Combination',c,ne)
    #print('AUC',auc)

    if auc > best_score:
        best_model=gb
        best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(X_train,Y_train)
preds=gb.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train,Y_train)
preds = xgb_model.predict(eva_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



LOGISTIC REGRESSION
2389 0 611 0
0.0
1.0
0.5
Decision Tree
2076 313 536 75
0.12274959083469722
0.8689828380075345
0.4958662144211159
LGBM Classifier
2349 40 602 9
0.014729950900163666
0.9832565927166178
0.49899327180839076
XGB Classifier
2355 34 605 6
0.009819967266775777
0.9857681038091252
0.4977940355379505


## Poison Attack

In [600]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_train = pd.concat([d1],ignore_index = True)

In [601]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, random_state=4)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(eva_train,Y_train)

preds = lr.predict(X_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(eva_train,Y_train)

preds =logi.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
# parameters = {'criterion' : ["gini", "entropy"],
#               'n_estimators': [25,50,100,150,175,200]}
# best_model=None
# best_score=0.0

# for ne in parameters['n_estimators']:
#     gb = LGBMClassifier(n_estimators=ne)
#     gb.fit(X_train,Y_train)
#     y_predg=gb.predict(X_test)

#     accuracy=accuracy_score(Y_test,y_predg)
#     auc=roc_auc_score(Y_test,y_predg)
#     # print('Combination',c,ne)
#     #print('AUC',auc)

#     if auc > best_score:
#         best_model=gb
#         best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(eva_train,Y_train)
preds=gb.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(eva_train,Y_train)
preds = xgb_model.predict(X_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



LOGISTIC REGRESSION
2416 0 584 0
0.0
1.0
0.5
Decision Tree
1491 925 360 224
0.3835616438356164
0.617135761589404
0.5003487027125102
LGBM Classifier
2358 58 569 15
0.025684931506849314
0.9759933774834437
0.5008391544951465
XGB Classifier
2175 241 484 100
0.17123287671232876
0.9002483443708609
0.5357406105415948


In [602]:
## Evasion Attack

x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(testloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_test = pd.concat([d1],ignore_index = True)

DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, stratify=y, random_state=42)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train,Y_train)

preds = lr.predict(eva_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(X_train,Y_train)

preds =logi.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
parameters = {'criterion' : ["gini", "entropy"],
              'n_estimators': [25,50,100,150,175,200]}
best_model=None
best_score=0.0

for ne in parameters['n_estimators']:
    gb = LGBMClassifier(n_estimators=ne)
    gb.fit(X_train,Y_train)
    y_predg=gb.predict(X_test)

    accuracy=accuracy_score(Y_test,y_predg)
    auc=roc_auc_score(Y_test,y_predg)
    # print('Combination',c,ne)
    #print('AUC',auc)

    if auc > best_score:
        best_model=gb
        best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(X_train,Y_train)
preds=gb.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train,Y_train)
preds = xgb_model.predict(eva_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)





## Poison Attack

x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_train = pd.concat([d1],ignore_index = True)

DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, random_state=42)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(eva_train,Y_train)

preds = lr.predict(X_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(eva_train,Y_train)

preds =logi.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
# parameters = {'criterion' : ["gini", "entropy"],
#               'n_estimators': [25,50,100,150,175,200]}
# best_model=None
# best_score=0.0

# for ne in parameters['n_estimators']:
#     gb = LGBMClassifier(n_estimators=ne)
#     gb.fit(X_train,Y_train)
#     y_predg=gb.predict(X_test)

#     accuracy=accuracy_score(Y_test,y_predg)
#     auc=roc_auc_score(Y_test,y_predg)
#     # print('Combination',c,ne)
#     #print('AUC',auc)

#     if auc > best_score:
#         best_model=gb
#         best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(eva_train,Y_train)
preds=gb.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(eva_train,Y_train)
preds = xgb_model.predict(X_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)





LOGISTIC REGRESSION
2389 0 611 0
0.0
1.0
0.5
Decision Tree
1865 524 467 144
0.23567921440261866
0.7806613645876936
0.5081702894951562
LGBM Classifier
2353 36 603 8
0.01309328968903437
0.9849309334449561
0.4990121115669952
XGB Classifier
2353 36 599 12
0.019639934533551555
0.9849309334449561
0.5022854339892538
LOGISTIC REGRESSION
2416 0 584 0
0.0
1.0
0.5
Decision Tree
981 1435 229 355
0.6078767123287672
0.4060430463576159
0.5069598793431915
LGBM Classifier
2318 98 562 22
0.03767123287671233
0.9594370860927153
0.4985541594847138
XGB Classifier
2167 249 501 83
0.1421232876712329
0.8969370860927153
0.519530186881974
